In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go
import pandas as pd
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
exp_imp_completa_UF_df = pd.read_csv("data/exp_imp_completa_UF_df.csv")
#exp_imp_completa_UF_df

In [3]:
exp_imp_br_df = pd.read_csv('data/exp_imp_br.csv')
#exp_imp_br_df

In [4]:
app = dash.Dash()

n_top_options = []
for n_top in range(1,28):
    n_top_options.append({'label': str(n_top), 'value': n_top})
    
min_year = exp_imp_completa_UF_df['Ano'].min()
max_year = exp_imp_completa_UF_df['Ano'].max()

app.layout = html.Div([

    #html.H1("Balança Comercial Brasileira"),
    
    html.Div([
       
        dcc.Graph(id='output-exportacao'),
        
         dcc.Markdown('Período'),
        dcc.RangeSlider(
        id='range-slider',
        min=min_year, max=max_year, step=1,
        marks={1997: str(min_year), 2021: str(max_year)},
        value=[min_year, max_year]
        ),
        
        dcc.Markdown('N Estados que mais exportam') ,
        dcc.Dropdown(id='year_picker', options=n_top_options, value=10)
        ], style={'width': '25%','display': 'inline-block', 'vertical-align': 'middle'}),
    
    html.Div([
        dcc.Graph(id='top_N'),
        ], style={'width': '75%', 'display': 'inline-block', 'vertical-align': 'top'}) ,
    
    html.Div([
        dcc.Graph(id='serie_temporal'),
        
        ], style={'width': '100%', 'display': 'inline-block', 'vertical-align': 'top'})      
])

In [5]:
@app.callback(Output('serie_temporal', 'figure'),
              Output('output-exportacao', 'figure'),
              Input('range-slider', 'value'))

def serie_temporal_indicador(slider_range):
    low, high = slider_range
    mask = (exp_imp_br_df['Ano'] >= low) & (exp_imp_br_df['Ano'] <= high)
    
    total_exp = exp_imp_br_df[mask]["Exportações"].sum()
    total_imp = exp_imp_br_df[mask]["Importações"].sum()
    
    fig_serie_temporal = go.Figure( 
    data =[go.Scatter(x=exp_imp_br_df[mask]["Ano"], y=exp_imp_br_df[mask]["Exportações"],
                      mode='lines+markers', name='Exportações'),

           go.Scatter(x=exp_imp_br_df[mask]["Ano"], y=exp_imp_br_df[mask]["Importações"],
                      mode='lines+markers', name='Importações')],
            layout=go.Layout(
            title=go.layout.Title(text="Exportações e importações entre os anos " +str(low)+" e "+str(high))
            )
    )

    fig_serie_temporal.update_xaxes(tickangle = 45,
    showgrid=False,
    dtick="M1")

    fig_serie_temporal.update_yaxes(showgrid=True,
                tickprefix="US$ ",
                ticksuffix="ilhões")
    
    fig_serie_temporal.update_layout(height=300)#width=400
   
    fig_indicador = go.Figure(go.Indicator(
                        mode = "number+delta",
                        value = total_exp ,
                        delta = {"reference": total_imp,'position': "top"},
                        title = {"text": "Saldo"},
                        domain = {'y': [0, 1], 'x': [0, 1]}))

    fig_indicador.update_layout(height=250)
    
    return fig_serie_temporal, fig_indicador

In [6]:
@app.callback(Output('top_N', 'figure'),
              Input('range-slider', 'value'),
              Input('year_picker', 'value'))

def top_N(slider_range, n_top):
    low, high = slider_range
    mask = (exp_imp_completa_UF_df['Ano'] >= low) & (exp_imp_completa_UF_df['Ano'] <= high)
    
    # O dataFrame é filtrado aqui
    top_N_exp = (exp_imp_completa_UF_df[mask][["Estado", "Exportação"]].groupby(["Estado"]).sum()
               ).sort_values(by=["Exportação"],ascending=False)[:n_top]

    top_N_exp['Estado'] = top_N_exp.index
    
    top_N_exp = top_N_exp.sort_values(by=["Exportação"])

    fig_top_N = px.bar(x=top_N_exp["Exportação"], y=top_N_exp["Estado"], orientation='h'
                 ,title=("Top "+ str(n_top)+ " estados exportadores entre os anos " +str(low)+" e "+str(high)))
        
    fig_top_N.update_layout(height=490)
    return fig_top_N

In [8]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Oct/2021 19:11:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 19:11:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 19:11:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 19:11:25] "GET /_dash-component-suites/dash_core_components/async-markdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 19:11:25] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 19:11:25] "GET /_dash-component-suites/dash_core_components/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 19:11:25] "GET /_dash-component-suites/dash_core_components/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 19:11:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 19:11:25] "GET /_dash-component-suites/dash_core_components/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 19:11:25] "POST /_d